# GroupEngAssign

> Invoke Package Group Eng to Assign Student in Groups

In [ ]:
#| default_exp assign

In [ ]:
#| hide
from nbdev.showdoc import *
import os
import shutil
import pandas as pd

In [ ]:
#| export
import GroupEng

def assign_groups(groupeng_config:str # Directory for the GroupEng config yml file
                 ) -> (bool, str): # Status and output directory of the compile file.
    status, outdir = GroupEng.run(groupeng_config)
    return status, outdir 

In [ ]:
%%capture
status, outdir = assign_groups("../data/195_group_specification.groupeng")

In [ ]:
# take a look at the output
status, outdir

(False,
 '/Users/scottyang/Desktop/SP23_Working/Group_Eng/CanvasGroupy/nbs/data/groups_195_group_specification_2023-05-11_15-49-41')

The false means that at least one requirment is not satisfied. We can take a look at the file that was generated.

In [ ]:
details = pd.read_csv(os.path.join(outdir, "195_group_specification_classlist.csv"), on_bad_lines='skip')
details.head()

,ID,GPA,Gender,Ethnicity,Major,Math Skills,Programming Skills,Statistic Skills,Group Number
0,dol005,3.9,M,-,COGS,5,5,3,1
1,xiw013,3.1,M,-,DSC,3,1,5,1
2,phantom,None,None,None,None,None,None,None,1
3,nmackler,3.4,F,B,COGS,2,3,2,2
4,jiz100,2.5,M,H,MATH,2,2,1,2


The following package will generate Canvas and GitHub groups based on this generated csv files.

In [ ]:
# | hide
# keep-track of the file generated for clean up
with open("dir.txt", "w+") as f:
    f.write(outdir)

In [ ]:
#| hide 
shutil.rmtree(outdir)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()